# **Music genre classification**

License : CC BY-NC-SA 4.0

The purpose of this tutorial is to train a model to automatically classify music according to its genre.


For this, we will use a database (GTZAN) which will be used to train several models. 

We will use the PyTorch framework and its extensions torchaudio/torchvision. 

Computations will be hardware accelerated by the use of Jean-Zay GPUs.


This notebook allows us to explore and become familiar with the GTZAN dataset.

> **Let's check the loaded modules and the presence of the reserved GPU(s): the notebook need to be launched from a compute node and not a front-end to take advantage of the GPUs.**


> **The "!nvidia-smi" command allows quickly visualizing the status of the GPUs.**

In [ ]:
!module list
!conda env list
!nvidia-smi
from torch.cuda import device_count
print('Torch detecting {0} GPUs compatible with CUDA'.format(device_count()))

## **Imports**

Multiple libraries will be used throughout the course to carry out data transformations, file management, visualization, etc.

> **We use the pytorch-gpu/py3/1.11.0 environment which contains all the libraries useful for this project.**

> **You can modify the imported scripts and re-import them without restarting the kernel thanks to the autoreload command.**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np # Maths
import pandas as pd #Data analysis and manipulation

import os # Operating System interaction

import matplotlib # Visualization
import matplotlib.pyplot as plt
#Necessary to view the figures in the notebook
%matplotlib inline 

from tqdm import tqdm #progress bar


import torch
from torch import optim
from torch import nn

> **Warning: Windows users must install soundfile (pip install soundfile) and add the following line :**

> torchaudio.set_audio_backend("sox_io")

In [ ]:
# Set randomness for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Some variables for file management
project_dir = os.getcwd()
data_dir = '/mnt/datasets/TP_Methodologie/GTZAN/data'
figures_dir = os.path.join(project_dir, 'figures')
scripts_dir = os.path.join(project_dir, 'scripts')
models_dir = os.path.join(project_dir, 'models')
dataset_dir = os.path.join(data_dir, 'raw')

---
---
---
---
---

# **Exploratory Data Analysis**

**Before using the database to train a model, it is important to analyse it. This step is called EDA for Exploratory Data Analysis. The objectives are multiple:**
- Understand the data
- Clean up the database
- Identify relationships/dependencies

> Taking the time to get to know the data is essential. It is possible to anticipate several problems even before training the model.


**Several actions can be taken to :*
- Remove corrupted data that may generate bugs when used
- Remove redundant data that weighs down the database
- Detect outliers that may be detrimental to training
- Identify imbalances between classes and decide on a strategy to counteract them
- Understand the complexity of the data and the problem


In order to carry out an EDA, it is often necessary to use a variety of tools:
- Visualization tools to help interpret the data
- Statistical tools to identify some properties of the database
- Dimensionality reduction to compress the information, identify outliers, visualise relationships between classes for example



## **The dataset**

We import the GTZAN database (http://marsyas.info/downloads/datasets.html) using a modified version of the class proposed by Torchaudio.
This allows us to take advantage of the corrections made to the original version: removal of duplicates, division of the database into sets for training, validation and testing steps.
A modification has been made to allow the application of transformations to the database at load time.

Moreover, the generated class corresponds to the standards of the "Dataset" and "DataLoader" classes of Pytorch. This enables us to efficiently take advantage of the methods associated with these classes.


In [ ]:
!pwd

In [ ]:
from scripts.dataset import GTZAN
from scripts.utils import list_classes
dataset = GTZAN(root=dataset_dir,download=False)
genres_dir = os.path.join(dataset_dir, 'genres')
classes = list_classes(genres_dir)
print(type(dataset))
print('The loaded dataset contains {} different music samples.'.format(len(dataset)))
print('It is divided into {} genres with 100 extracts each.'.format(len(classes)))

> **Uncomment the next line to listen to one of the clips if the environment allows it.**
> **For the remainder of this notebook, we will use the training dataset corrected by TorchAudio.** This version has already removed redundant or damaged data from the database.
> You can also change "training" to "validation" or "testing" to see the sizes and elements of those sets.


In [ ]:
dataset = GTZAN(root=dataset_dir, subset='training', download=True)
print('Le dataset chargé comporte {} extraits musicaux différents.'.format(len(dataset)))

> **What is the total size of the filtered dataset?**

## **Audio data**

The musical extracts are all Mono audio files, 16 bits sampled at 22050 Hz available in .wav format.

The dataset allows retrieving for each extract the waveform, its sampling rate and its genre for each extract.

The waveform is the variation of air pressure (acoustic wave) over time.

In [ ]:
track = 0
waveform,samplerate,label,target= dataset[track]
print(" dimensions de l'oscillogramme {} \n échantillonnage : {} \n genre : {} ".format(waveform.size(),samplerate,label))

> **Uncomment the following line to listen to one of the excerpts if the environment allows it.**

> **Why is the sampling rate important? Try to give the audio player a samplerate multiplied by a factor.**

In [ ]:
import IPython
IPython.display.Audio(waveform,rate=samplerate)

In [ ]:
from scripts.utils import check_sizes
sizes_uniformity, sizes = check_sizes(dataset)

Each signal corresponds to a set of amplitudes measured over time at a sampling rate of 22050 Hz.

> **It can be identified that not all recordings have exactly the same duration, which may cause problems for some functions. We can already anticipate some future errors and think about possible solutions.**

### **Visualization**

It is easily possible to visualize this signal by displaying the evolution of the amplitude at each sample (time).

In [ ]:
#import librosa.display
plt.figure(figsize=(13, 8))
#librosa.display.waveshow(waveform.numpy().reshape((-1,)), sr=samplerate)
plt.title(f"Music waveform (sampling rate)  : {samplerate}Hz)")
plt.xlabel("samples")
plt.ylabel("amplitude")
plt.plot(waveform.t().numpy())

To classify musical excerpts, it is possible to try to use the signal in its original version.

However, this is generally not recommended. Such a signal does not effectively highlight the useful features for identifying the genre.

There are two methods that can be used together:
- Finding an effective model for our type of data
- Highlighting features in a way that simplifies the exercise for the classifier

---
---
---
---
---

## **Features extraction**

From this signal, it is possible to :
- Apply transformations that modify the signal
- Extract information from the signal

Some transformations can be beneficial to the task to accomplish. For example, normalizing the amplitudes between extracts to compensate for differences in intensity during capture can be beneficial.





### **Specifically selected and constructed features**

There is a wide variety of features that can be extracted from a sound signal. These range from relatively simple concepts such as the number of zero-crossing rates, to frequency statistics over the entire musical sample.

Many features are based on Fourier transforms. These enable a signal to be broken down into its component frequencies. In order to classify music, it is common to apply statistics to these features in order to have a compact descriptor of information selected with the help of musical knowledge. We will find information such as the tempo, tonnetz and harmonics in it.

> To extract these features, there are several libraries including Librosa: https://librosa.org/doc/latest/index.html whose documentation can be consulted for more information on each feature. 

In [ ]:
features_path = os.path.join(data_dir, 'features', 'features_30_sec.csv')
data = pd.read_csv(features_path)
data.head()

> **The features files 'features_30_sec.csv' and 'features_3_sec.csv' contain the means and variances for each extract of the following 30 features:**

In [ ]:
for col in data.columns:
    print(col)

> It is possible to construct a descriptor from features and use it for classification. The correlation between them can help in feature selection.

> **For example, look at the correlation between the variances. Then test the correlation between the means. To do so, change the stat parameter in the compute_correlation_matrix function.**

In [ ]:
# Computing the Correlation Matrix
from scripts.utils import compute_correlation_matrix
corr = compute_correlation_matrix(data, stat='var', show = True) # Choose stat between mean and var

> **You can try to confirm the choice of certain features by viewing the distribution of classes for a feature. Use the feature parameter in the boxplot function to check which classes you may have found interesting from the correlations above.**

In [ ]:
from scripts.utils import boxplot
boxplot(data, feature='perceptr_var')

> A manual selection is usually tedious. There are methods to reduce the dimensionality of the data while retaining the maximum amount of information.

In [ ]:
from scripts.utils import pca
pca(data)

> **The PCA makes it possible to visualise which classes are most similar and thus to anticipate which classes will pose difficulties. A solution can then be sought, possibly the features that distinguish these classes.**

## **Baseline for model performance**

With these data, it is totally possible to train a first simple classifier in order to have a performance baseline.

> **By default, all the numerical features are used, but you can also select some yourself.**

In [ ]:
from sklearn import preprocessing

features = data.loc[:, (data.columns != 'filename') * (data.columns != 'label')] #entrées : Ne pas sélectionner les noms de fichiers ou les labels
labels = data['label'] # sorties


cols_names = features.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)

# new data frame with the new scaled data. 
features = pd.DataFrame(np_scaled, columns = cols_names)
features.head()

In order to evaluate our model, we need to separate the dataset into a training and a test set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=2021)

> **A large selection of algorithms are available in popular libraries such as sklearn. For a dataset of this size with a small number of features, it is possible to train a number of Machine Learning models quickly enough to compare their performances. You can find the classifiers available with sklearn using this link: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning . You should only need to change the "model" variable and import the class.**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model = SVC()
model.fit(X_train, y_train)
preds = model.predict(X_test)
#print(confusion_matrix(y_test, preds))
print('Model Accuracy' ':', round(accuracy_score(y_test, preds), 5), '\n')

In [ ]:
# Confusion Matrix
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sn


gtzan_genres = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]


cm = sklearn.metrics.confusion_matrix(y_test, preds, normalize='true') #normalize='true' 
df_cm = pd.DataFrame(cm, index = [i for i in gtzan_genres],
                  columns = [i for i in gtzan_genres])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

>**Try training a model using the features on 3-second clips. To do this, change the line:**


>**features_path = os.path.join(data_dir, 'features', 'features_30_sec.csv'** with **features_3_sec.csv**

### **More generalist representations**

Rather than trying to describe the musical extract as a whole, it is natural to want to preserve the sequential aspect of the music.

A common technique for doing this is to apply a frequency representation to a sliding window smaller than the musical extract to represent the evolution of frequencies over time.
There is a wide variety of representations that apply different quantizations and sampling as well as different scaling to take into account, for example, differences in human perception of intensity according to frequency.

Although these representations are also available from Librosa, we will use Torchaudio for the increased performance and compatibility with Pytorch and Torchvision (Tensor type compatibility).

> **crop_audio permet de mettre tous les extraits à la même durée. Vous aurez besoin de tvt.Lambda pour l'intégrer dans votre transformation.**


> **Use this section to define the transformations you wish to apply to the database. You should aim to retain the temporal aspect of the audio clip, while compressing the information to approximate the data used in the design of the LeNet and VGG models (256*256 colour images).**

> Below, you can modify the transformations applied when loading the data and view them.

> Performance may vary depending on the features selected and passed to the models. Various features and representations can be easily selected with torchaudio.transforms or torchvision.transforms.
> Useful transformations are available with https://pytorch.org/audio/0.11.0/transforms.html and https://pytorch.org/vision/0.12/transforms.html.

> Changing features, modifying them by normalizing them, or applying various transforms significantly affects performance.


> torch.nn.Sequential and torchvision.transforms.Compose allow transforms to be combined.

> Use **tvt.Lambda** to add transforms not coming from the torchaudio or torch vision modules.

> Caution: Not all inputs have the same size. All outputs must have the same size.

> **crop_audio allows you to set all clips to the same duration. You will need tvt.Lambda to integrate it into your transformation. You will need tvt.Lambda to integrate it into your transformation.**

> Some useful functions: 
> - torchvision.transforms.Lambda
> - torchvision.transforms.Crop
> - torchvision.transforms.Pad
> - torchvision.transforms.Resize
> - torchaudio.transforms.Spectrogram
> - torchaudio.transforms.AmplitudeToDB
> - torchaudio.transforms.MelScale
> - torchaudio.transforms.MelSpectrogram
> - torchaudio.transforms.MFCC

In [ ]:
from torch import nn
import torchaudio.transforms as tat
import torchvision.transforms as tvt
import torchvision
from scripts.utils import crop_audio
#from torchvision import transforms, utils
#from torchvision.transforms import ToTensor, Lambda, Resize
#Crop = tvt.Lambda(lambda waveform: crop_audio(waveform, duration = 20, augment=False))
transformation = tvt.Compose([tat.Spectrogram(), tvt.Resize(size=[201, 3306])])
dataset = GTZAN(root=dataset_dir, download=True, transform=transformation)

In [ ]:
from scripts.utils import check_sizes
sizes_uniformity, sizes = check_sizes(dataset)

In [ ]:
features = dataset[0][0].log2().numpy().squeeze()
plt.figure(figsize=(30, 8))
plt.title("Spectrogramme de l'extrait musical")
plt.imshow(features,cmap='inferno') 
plt.colorbar()

> If you are colourblind, try another cmap: https://matplotlib.org/stable/tutorials/colors/colormaps.html


Note: transformations are applied when the element is called in the database and not when it is defined.

---
---
---
---
---

# To free up some memory, you can reboot the kernel, especially if you switch to the next notebook.